# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# Imports
import os
import json
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import Markdown, display

In [ ]:
# Load environment variables and check API keys

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set (optional)")

In [ ]:
# Initialize model clients

openai = OpenAI()

# Set up alternative model endpoints
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url) if anthropic_api_key else None
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url) if google_api_key else None
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

# Model mapping
MODELS = {
    "GPT-4.1-mini": {"client": openai, "model": "gpt-4.1-mini"},
    "Claude Sonnet 4.5": {"client": anthropic, "model": "claude-sonnet-4-5-20250929"},
    "Gemini 2.5 Flash": {"client": gemini, "model": "gemini-2.5-flash-lite"},
    "Llama 3.2 (Local)": {"client": ollama, "model": "llama3.2"}
}

print("Model clients initialized successfully!")

In [ ]:
# System prompt for SQL expert

system_prompt = """You are an expert SQL database engineer with deep knowledge of SQL syntax, optimization, and best practices.

Your role is to:
1. Generate accurate, efficient SQL queries from natural language questions
2. Use only the tables and columns provided in the database schema
3. Follow standard SQL syntax (PostgreSQL/SQLite compatible)
4. Provide clear explanations of your queries
5. Suggest optimizations when relevant

When responding:
- Return the SQL query in a fenced code block (```sql)
- Add a brief explanation of what the query does
- If the question is ambiguous, make reasonable assumptions and state them
- If a schema is not provided, generate generic SQL with common table/column names

Always prioritize correctness and clarity."""

In [ ]:
# Streaming SQL generation function

def generate_sql_stream(question, schema, model_name):
    """
    Generate SQL query from natural language with streaming response
    """
    model_info = MODELS.get(model_name)
    if not model_info or model_info["client"] is None:
        yield "Error: Selected model is not available. Please check your API keys."
        return
    
    client = model_info["client"]
    model = model_info["model"]
    
    user_content = f"Database Schema:\n{schema.strip()}\n\nQuestion: {question}" if schema.strip() else f"Question: {question}"
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content}
    ]
    
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True
        )
        
        result = ""
        for chunk in stream:
            delta = chunk.choices[0].delta.content or ""
            result += delta
            yield result
            
    except Exception as e:
        yield f"Error: {str(e)}"

In [ ]:
# BONUS: Tool for executing SQL queries

DB = "sql_demo.db"

def execute_sql_query(query):
    """
    Execute a SQL query against a demo database
    Returns the results as a formatted string
    """
    print(f"TOOL CALLED: Executing SQL query", flush=True)
    try:
        with sqlite3.connect(DB) as conn:
            cursor = conn.cursor()
            cursor.execute(query)
            results = cursor.fetchall()
            
            if not results:
                return "Query executed successfully. No results returned."
            
            column_names = [description[0] for description in cursor.description]
            result_str = f"Results ({len(results)} rows):\n\n"
            result_str += " | ".join(column_names) + "\n"
            result_str += "-" * (len(result_str) - 1) + "\n"
            
            for row in results:
                result_str += " | ".join(str(val) for val in row) + "\n"
            
            return result_str
    except Exception as e:
        return f"Error executing query: {str(e)}"

# Initialize demo database with sample data
def init_demo_db():
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        
        cursor.execute('DROP TABLE IF EXISTS orders')
        cursor.execute('DROP TABLE IF EXISTS customers')
        cursor.execute('DROP TABLE IF EXISTS products')
        
        cursor.execute('''
            CREATE TABLE customers (
                id INTEGER PRIMARY KEY,
                name TEXT,
                email TEXT,
                created_at DATE
            )
        ''')
        
        cursor.execute('''
            CREATE TABLE orders (
                id INTEGER PRIMARY KEY,
                customer_id INTEGER,
                total REAL,
                order_date DATE,
                FOREIGN KEY (customer_id) REFERENCES customers(id)
            )
        ''')
        
        cursor.execute('''
            CREATE TABLE products (
                id INTEGER PRIMARY KEY,
                name TEXT,
                price REAL
            )
        ''')
        
        cursor.executemany('INSERT INTO customers VALUES (?, ?, ?, ?)', [
            (1, 'Alice Johnson', 'alice@email.com', '2025-01-15'),
            (2, 'Bob Smith', 'bob@email.com', '2025-12-20'),
            (3, 'Carol White', 'carol@email.com', '2026-02-01'),
            (4, 'David Brown', 'david@email.com', '2026-02-15')
        ])
        
        cursor.executemany('INSERT INTO orders VALUES (?, ?, ?, ?)', [
            (1, 1, 150.00, '2026-02-01'),
            (2, 1, 200.00, '2026-02-15'),
            (3, 2, 75.00, '2025-12-25'),
            (4, 3, 300.00, '2026-02-20'),
            (5, 4, 450.00, '2026-02-28')
        ])
        
        cursor.executemany('INSERT INTO products VALUES (?, ?, ?)', [
            (1, 'Laptop', 999.99),
            (2, 'Mouse', 29.99),
            (3, 'Keyboard', 79.99),
            (4, 'Monitor', 299.99)
        ])
        
        conn.commit()
    print("Demo database initialized with sample data!")

init_demo_db()

In [ ]:
# Tool definition for SQL execution

execute_sql_function = {
    "name": "execute_sql_query",
    "description": "Execute a SQL query against the demo database and return the results. Use this when the user wants to run or test a generated SQL query.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The SQL query to execute"
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": execute_sql_function}]

In [ ]:
# Enhanced chat function with tool calling support

def handle_tool_calls(message):
    """Handle tool calls from the LLM"""
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "execute_sql_query":
            arguments = json.loads(tool_call.function.arguments)
            query = arguments.get('query')
            result = execute_sql_query(query)
            responses.append({
                "role": "tool",
                "content": result,
                "tool_call_id": tool_call.id
            })
    return responses

def chat_with_tools(question, schema, model_name, enable_tools):
    """
    Chat function that supports tool calling for SQL execution
    """
    model_info = MODELS.get(model_name)
    if not model_info or model_info["client"] is None:
        yield "Error: Selected model is not available. Please check your API keys."
        return
    
    client = model_info["client"]
    model = model_info["model"]
    
    user_content = f"Database Schema:\n{schema.strip()}\n\nQuestion: {question}" if schema.strip() else f"Question: {question}"
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content}
    ]
    
    try:
        if enable_tools:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools,
                stream=False
            )
            
            while response.choices[0].finish_reason == "tool_calls":
                message = response.choices[0].message
                tool_responses = handle_tool_calls(message)
                messages.append(message)
                messages.extend(tool_responses)
                response = client.chat.completions.create(
                    model=model,
                    messages=messages,
                    tools=tools,
                    stream=False
                )
            
            yield response.choices[0].message.content
        else:
            stream = client.chat.completions.create(
                model=model,
                messages=messages,
                stream=True
            )
            
            result = ""
            for chunk in stream:
                delta = chunk.choices[0].delta.content or ""
                result += delta
                yield result
            
    except Exception as e:
        yield f"Error: {str(e)}"

In [ ]:
# BONUS: Audio input/output functions

def transcribe_audio(audio_file):
    """Convert audio input to text using Whisper"""
    if audio_file is None:
        return ""
    
    try:
        with open(audio_file, "rb") as f:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file=f
            )
        return transcript.text
    except Exception as e:
        print(f"Audio transcription error: {str(e)}")
        return ""

def text_to_speech(text):
    """Convert text response to speech"""
    try:
        response = openai.audio.speech.create(
            model="gpt-4o-mini-tts",
            voice="alloy",
            input=text
        )
        return response.content
    except Exception as e:
        print(f"Text-to-speech error: {str(e)}")
        return None

In [ ]:
# Gradio UI - Full-featured SQL Generator with Audio Support

def process_query(audio_input, text_input, schema, model_name, enable_tools, enable_audio_output):
    """
    Main processing function that handles both audio and text input
    """
    question = text_input
    
    if audio_input is not None:
        transcribed = transcribe_audio(audio_input)
        if transcribed:
            question = transcribed
    
    if not question.strip():
        yield "Please provide a question either via text or audio.", None
        return
    
    result = ""
    for chunk in chat_with_tools(question, schema, model_name, enable_tools):
        result = chunk
        yield chunk, None
    
    if enable_audio_output:
        audio = text_to_speech(result)
        yield result, audio
    else:
        yield result, None

# Default schema
default_schema = """Table: customers (id, name, email, created_at)
Table: orders (id, customer_id, total, order_date)
Table: products (id, name, price)"""

# Build the UI
with gr.Blocks(title="SQL Query Generator") as demo:
    gr.Markdown("# 🗄️ Natural Language to SQL Query Generator")
    gr.Markdown("Ask questions in natural language and get SQL queries generated by AI. Optionally use your voice!")
    
    with gr.Row():
        with gr.Column(scale=2):
            audio_input = gr.Audio(
                sources=["microphone"],
                type="filepath",
                label="🎤 Voice Input (Optional)"
            )
            text_input = gr.Textbox(
                label="💬 Text Input",
                placeholder="e.g., List all customers who placed an order in the last 30 days",
                lines=3
            )
            schema_input = gr.Textbox(
                label="📊 Database Schema (Optional)",
                value=default_schema,
                lines=5,
                placeholder="Describe your database tables and columns"
            )
            
            with gr.Row():
                model_selector = gr.Dropdown(
                    choices=list(MODELS.keys()),
                    value="GPT-4.1-mini",
                    label="🤖 Select Model"
                )
                enable_tools = gr.Checkbox(
                    label="🔧 Enable SQL Execution",
                    value=False,
                    info="Allow the AI to execute queries on demo database"
                )
                enable_audio = gr.Checkbox(
                    label="🔊 Audio Response",
                    value=False,
                    info="Get spoken response"
                )
            
            submit_btn = gr.Button("Generate SQL Query", variant="primary")
        
        with gr.Column(scale=3):
            output = gr.Markdown(label="📝 Generated SQL & Explanation")
            audio_output = gr.Audio(label="🔊 Audio Response", autoplay=True)
    
    gr.Examples(
        examples=[
            [None, "List all customers who placed an order in the last 30 days, with their total spend", default_schema, "GPT-4.1-mini", False, False],
            [None, "Show me the top 5 most expensive products", default_schema, "GPT-4.1-mini", False, False],
            [None, "Find customers who haven't placed any orders", default_schema, "Claude Sonnet 4.5", False, False],
            [None, "What is the average order value per customer?", default_schema, "GPT-4.1-mini", True, False],
        ],
        inputs=[audio_input, text_input, schema_input, model_selector, enable_tools, enable_audio]
    )
    
    submit_btn.click(
        fn=process_query,
        inputs=[audio_input, text_input, schema_input, model_selector, enable_tools, enable_audio],
        outputs=[output, audio_output]
    )
    
    text_input.submit(
        fn=process_query,
        inputs=[audio_input, text_input, schema_input, model_selector, enable_tools, enable_audio],
        outputs=[output, audio_output]
    )

demo.launch(inbrowser=True)

In [ ]:
# Alternative: Simple streaming interface without tools

def simple_generate(question, schema, model_name):
    """Simple streaming SQL generation without tool calling"""
    yield from generate_sql_stream(question, schema, model_name)

# Uncomment below to use the simpler interface without tool calling:

# question_input = gr.Textbox(
#     label="Your Question:",
#     placeholder="e.g., Show all orders from the last week",
#     lines=3
# )
# schema_input = gr.Textbox(
#     label="Database Schema:",
#     value=default_schema,
#     lines=5
# )
# model_selector = gr.Dropdown(
#     choices=list(MODELS.keys()),
#     value="GPT-4.1-mini",
#     label="Select Model"
# )
# output = gr.Markdown(label="Generated SQL:")

# view = gr.Interface(
#     fn=simple_generate,
#     title="SQL Query Generator",
#     inputs=[question_input, schema_input, model_selector],
#     outputs=[output],
#     examples=[
#         ["List all customers", default_schema, "GPT-4.1-mini"],
#         ["Show orders from last month", default_schema, "Claude Sonnet 4.5"]
#     ],
#     flagging_mode="never"
# )
# view.launch(inbrowser=True)

# 📚 How to Use This Application

## Features Implemented:

### ✅ Core Requirements:
1. **Gradio UI**: Beautiful, intuitive interface with multiple input options
2. **Streaming**: Real-time response generation for better UX
3. **System Prompt**: Expert SQL engineer persona with detailed instructions
4. **Model Selection**: Switch between GPT-4.1-mini, Claude Sonnet 4.5, Gemini 2.5 Flash, and Llama 3.2 (local)

### ✅ Bonus Features:
5. **Tool Calling**: Enable SQL execution against a demo database
6. **Audio Input**: Speak your question using the microphone
7. **Audio Output**: Get spoken responses from the AI

## How to Use:

### Basic Usage:
1. Type your question in natural language (or use the microphone)
2. Optionally modify the database schema
3. Select your preferred AI model
4. Click "Generate SQL Query"

### Advanced Features:
- **Enable SQL Execution**: Check this box to let the AI actually run queries on the demo database
- **Audio Response**: Check this box to hear the response spoken aloud
- **Voice Input**: Click the microphone icon to ask your question by speaking

## Example Questions:
- "List all customers who placed an order in the last 30 days"
- "What is the average order total?"
- "Show me customers with no orders"
- "Find the most expensive product"

## Demo Database Schema:
The demo database includes:
- **customers**: id, name, email, created_at
- **orders**: id, customer_id, total, order_date  
- **products**: id, name, price

Try enabling SQL execution to see real results from the demo database!

---

**Note**: Make sure you have your API keys set up in the `.env` file. At minimum, you need `OPENAI_API_KEY`. Other keys are optional.